In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Embedding, Flatten, Dot
from tensorflow.keras.models import Sequential,Model
from keras.layers import Input


In [14]:
df = pd.read_excel('datadummy2.xlsx')

In [15]:
print(df)

     Id User       Genre         Nama Usaha     Lokasi  Rating  \
0          1        Baju      Fashion Trend    Jakarta     4.2   
1          2     Makanan        Warung Sate  Tangerang     4.5   
2          3  Elektronik       Gadget Store      Bogor     4.1   
3          4    Aksesori     Accessorize It     Bekasi     4.3   
4          5  Kecantikan    Beauty Paradise      Depok     4.4   
..       ...         ...                ...        ...     ...   
898      899  Elektronik   Gadget Plus Plus    Jakarta     4.6   
899      900     Makanan      Restoran Cina  Tangerang     4.3   
900      901        Baju        Fashion Hub      Bogor     4.5   
901      902  Elektronik  Electronics Depot     Bekasi     4.2   
902      903     Makanan       Warung Bakmi    Jakarta     4.7   

             Nama Produk  Jumlah Follower  
0          Kemeja Casual           3500.0  
1       Sate Ayam Madura           4600.0  
2    Headphone Bluetooth           3800.0  
3      Gelang Kalung Set       

In [16]:
df['Nama Usaha'].duplicated()


0      False
1      False
2      False
3      False
4      False
       ...  
898     True
899    False
900     True
901    False
902    False
Name: Nama Usaha, Length: 903, dtype: bool

In [17]:
df = df.drop_duplicates(subset=['Nama Usaha'])

In [18]:
print(df)

     Id User       Genre         Nama Usaha     Lokasi  Rating  \
0          1        Baju      Fashion Trend    Jakarta     4.2   
1          2     Makanan        Warung Sate  Tangerang     4.5   
2          3  Elektronik       Gadget Store      Bogor     4.1   
3          4    Aksesori     Accessorize It     Bekasi     4.3   
4          5  Kecantikan    Beauty Paradise      Depok     4.4   
..       ...         ...                ...        ...     ...   
896      897     Makanan   Warung Sate Ayam     Bekasi     4.4   
897      898        Baju     Toko Baju Pria      Depok     4.1   
899      900     Makanan      Restoran Cina  Tangerang     4.3   
901      902  Elektronik  Electronics Depot     Bekasi     4.2   
902      903     Makanan       Warung Bakmi    Jakarta     4.7   

             Nama Produk  Jumlah Follower  
0          Kemeja Casual           3500.0  
1       Sate Ayam Madura           4600.0  
2    Headphone Bluetooth           3800.0  
3      Gelang Kalung Set       

In [19]:
print(df.isnull().sum())

Id User            0
Genre              0
Nama Usaha         0
Lokasi             0
Rating             0
Nama Produk        0
Jumlah Follower    2
dtype: int64


In [20]:
# Replace missing values with 0 in 'Jumlah Follower' column
df['Jumlah Follower'] = df['Jumlah Follower'].fillna(0)

# Print the updated DataFrame
print(df.isnull().sum())

Id User            0
Genre              0
Nama Usaha         0
Lokasi             0
Rating             0
Nama Produk        0
Jumlah Follower    0
dtype: int64


In [21]:
# Perform label encoding for categorical variables
label_encoder = LabelEncoder()
df['UserId'] = label_encoder.fit_transform(df['Id User'])
df['ProductId'] = label_encoder.fit_transform(df['Nama Produk'])

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the model architecture
num_users = len(df['UserId'].unique())
num_products = len(df['ProductId'].unique())

user_input = Input(shape=(1,))
product_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=num_users, output_dim=10)(user_input)
product_embedding = Embedding(input_dim=num_products, output_dim=10)(product_input)

user_embedding_flat = Flatten()(user_embedding)
product_embedding_flat = Flatten()(product_embedding)

dot_product = Dot(axes=1)([user_embedding_flat, product_embedding_flat])

model = Model(inputs=[user_input, product_input], outputs=dot_product)

# Compile and train the model
model.compile(optimizer='adam', loss='mse')
model.fit([train_df['UserId'], train_df['ProductId']], train_df['Rating'], epochs=10, verbose=1)

# Evaluate the model
loss = model.evaluate([test_df['UserId'], test_df['ProductId']], test_df['Rating'], verbose=1)
print('Test Loss:', loss)

# Select a random user from the test set
random_user_id = np.random.choice(test_df['UserId'].unique())

# Get the product ids
product_ids = test_df['ProductId'].unique()

predictions = []

# Make predictions for the random user
for product_id in product_ids:
    prediction = model.predict([np.array([random_user_id]), np.array([product_id])])[0][0]
    predictions.append((random_user_id, product_id, prediction))

# Sort predictions in descending order
predictions.sort(key=lambda x: x[2], reverse=True)

# Print top K recommendations for the random user
top_k = 3
print(f"\nTop {top_k} recommendations for User {random_user_id}:")
user_predictions = [(p_id, rating) for u_id, p_id, rating in predictions]
user_predictions.sort(key=lambda x: x[1], reverse=True)
for i in range(top_k):
    product_id = user_predictions[i][0]
    product_name = label_encoder.inverse_transform([product_id])[0]
    print(f"Recommendation {i+1}: {product_name}")

Epoch 1/10
19/19 [==============================] - 1s 2ms/step - loss: 19.9185
Epoch 2/10
19/19 [==============================] - 0s 2ms/step - loss: 19.9000
Epoch 3/10
19/19 [==============================] - 0s 2ms/step - loss: 19.8814
Epoch 4/10
19/19 [==============================] - 0s 2ms/step - loss: 19.8607
Epoch 5/10
19/19 [==============================] - 0s 2ms/step - loss: 19.8372
Epoch 6/10
19/19 [==============================] - 0s 2ms/step - loss: 19.8096
Epoch 7/10
19/19 [==============================] - 0s 2ms/step - loss: 19.7772
Epoch 8/10
19/19 [==============================] - 0s 2ms/step - loss: 19.7393
Epoch 9/10
19/19 [==============================] - 0s 2ms/step - loss: 19.6953
Epoch 10/10
5/5 [==============================] - 0s 3ms/step - loss: 20.0388
Test Loss: 20.038787841796875
1/1 [==============================] - 0s 22ms/step

Top 3 recommendations for User 346:
Recommendation 1: Mainan Edukatif
Recommendation 2: Car Parts and Accessories
Reco